# Population Data from CSV

This notebooks reads sample population data from `data/atlantis.csv` and plots it using Matplotlib. Edit `data/atlantis.csv` and re-run this cell to see how the plots change!

In [1]:
pip install "ray[rllib]" tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install swig


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gymnasium[box2d]


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install wandb


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import ray

ray.init()

2023-07-30 15:31:16,749	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-07-30 15:31:20,169	WARNING services.py:1832 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=2.03gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-07-30 15:31:20,842	INFO worker.py:1621 -- Started a local Ray instance.


Python version:,3.10.8
Ray version:,2.6.1


In [2]:
from ray import tune

tune.run("PPO",
         config={"env": "CartPole-v1",
                 "evaluation_interval": 2,
                 "evaluation_num_episodes": 20,
                 "num_workers": 1,
                 # other configurations go here, if none provided, then default configurations will be used
                 },
         local_dir="CartPole-v1",
         checkpoint_freq=2
         )

2023-07-30 15:31:30,240	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-07-30 15:31:30,725	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/codespace/.python/current/lib/python3.10/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
2023-07-30 15:31:30,727	WARNING syncer.py:260 -- You are using remote storage, but you don't have `fsspec` installed. This can lead to inefficient syncing behavior. To avoid this, install fsspec with `pip install fsspec`. Depending on your remote storage provider, consid

2023-07-30 15:32:00,015	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-07-30 15:32:00,016	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=9206) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=9206) 2023-07-30 15:32:06,152	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. Wh

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,evaluation,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_CartPole-v1_39a45_00000,24000,"{'ObsPreprocessorConnector_ms': 0.005588531494140625, 'StateBufferConnector_ms': 0.003810405731201172, 'ViewRequirementAgentConnector_ms': 0.2872037887573242}","{'num_env_steps_sampled': 24000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 24000, 'num_agent_steps_trained': 0}",{},140.14,{},500,140.14,13,8,"{'sampler_results': {'episode_reward_max': 500.0, 'episode_reward_min': 154.0, 'episode_reward_mean': 382.3, 'episode_len_mean': 382.3, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [190.0, 500.0, 500.0, 500.0, 500.0, 500.0, 459.0, 424.0, 500.0, 500.0, 337.0, 154.0, 290.0, 180.0, 500.0, 500.0, 228.0, 464.0, 200.0, 220.0], 'episode_lengths': [190, 500, 500, 500, 500, 500, 459, 424, 500, 500, 337, 154, 290, 180, 500, 500, 228, 464, 200, 220]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.477111249836208, 'mean_inference_ms': 1.5574585661734388, 'mean_action_processing_ms': 0.11841197055165967, 'mean_env_wait_ms': 0.05954083284351778, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.005270242691040039, 'StateBufferConnector_ms': 0.003560781478881836, 'ViewRequirementAgentConnector_ms': 0.27541518211364746}}, 'episode_reward_max': 500.0, 'episode_reward_min': 154.0, 'episode_reward_mean': 382.3, 'episode_len_mean': 382.3, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [190.0, 500.0, 500.0, 500.0, 500.0, 500.0, 459.0, 424.0, 500.0, 500.0, 337.0, 154.0, 290.0, 180.0, 500.0, 500.0, 228.0, 464.0, 200.0, 220.0], 'episode_lengths': [190, 500, 500, 500, 500, 500, 459, 424, 500, 500, 337, 154, 290, 180, 500, 500, 228, 464, 200, 220]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.477111249836208, 'mean_inference_ms': 1.5574585661734388, 'mean_action_processing_ms': 0.11841197055165967, 'mean_env_wait_ms': 0.05954083284351778, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.005270242691040039, 'StateBufferConnector_ms': 0.003560781478881836, 'ViewRequirementAgentConnector_ms': 0.27541518211364746}, 'num_agent_steps_sampled_this_iter': 7646, 'num_env_steps_sampled_this_iter': 7646, 'timesteps_this_iter': 7646, 'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}","{'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 9.890165373714748}, 'default_policy': {'total_loss': 9.890165373714748, 'policy_loss': -0.0023724655129277565, 'vf_loss': 9.891798198350203, 'vf_loss_unclipped': 4673.59326536264, 'vf_explained_var': -0.21216095065765544, 'entropy': 0.577543784687514, 'mean_kl_loss': 0.002465489803098089, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.15000000596046448}}, 'num_env_steps_sampled': 24000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 24000, 'num_agent_steps_trained': 0}",24000,0,24000,4000,278.351,0,0,0,0,1,0,0,0,"{'cpu_util_percent': 33.72727272727272, 'ram_util_percent': 44.41818181818182}",{},{},{},"{'mean_raw_obs_processing_ms': 0.5089938499795

2023-07-30 15:34:19,294	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-07-30 15:34:25,740	INFO tune.py:1148 -- Total run time: 175.02 seconds (139.28 seconds for the tuning loop).
2023-07-30 15:34:25,741	WARNING tune.py:1163 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)


In [3]:
from ray.rllib.algorithms.algorithm import Algorithm

algo = Algorithm.from_checkpoint("/workspaces/codespaces-jupyter/notebooks/CartPole-v1/PPO/PPO_CartPole-v1_39a45_00000_0_2023-07-30_15-32-00/checkpoint_000004")

2023-07-30 15:34:31,386	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-07-30 15:34:31,388	WARNING algorithm_config.py:656 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-07-30 15:34:31,390	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.e

In [ ]:
pip install moviepy

In [6]:
#from gymnasium.wrappers.record_video import RecordVideo

try:
    import gymnasium as gym

    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = gym.make("CartPole-v1",)
#env_name = gym.make("CartPole-v1", render_mode="rgb_array")
env = env_name
algo = PPOConfig().environment("CartPole-v1").rollouts(num_rollout_workers=1).build()

episode_reward = 0
terminated = truncated = False

if gymnasium:
    obs, info = env.reset()
else:
    obs = env.reset()

while not terminated and not truncated:
    action = algo.compute_single_action(obs)
    if gymnasium:
        obs, reward, terminated, truncated, info = env.step(action)
    else:
        obs, reward, terminated, info = env.step(action)
    episode_reward += reward
    print(obs)

2023-07-30 15:36:32,112	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-07-30 15:36:32,115	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/home/codespace/.python/current/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: Thi

[ 0.04824784 -0.24004433  0.00177911  0.2556794 ]
[ 0.04344695 -0.43519163  0.0068927   0.54892296]
[ 0.03474312 -0.6304097   0.01787116  0.84376955]
[ 0.02213492 -0.8257709   0.03474655  1.1420184 ]
[ 0.0056195  -1.0213293   0.05758692  1.4453924 ]
[-0.01480708 -0.8269612   0.08649477  1.1712447 ]
[-0.03134631 -1.0230948   0.10991966  1.489743  ]
[-0.0518082  -1.2193701   0.13971452  1.8146305 ]
[-0.07619561 -1.4157437   0.17600714  2.1472633 ]
[-0.10451048 -1.2227396   0.2189524   1.913709  ]


/home/codespace/.python/current/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
